(Visit the
[documentation](https://datafold-dev.gitlab.io/datafold/tutorial_index.html) page
to view the executed notebook.)

# Jointly Smooth Functions: An Example

For a detailed introduction see paper

Or Yair, Felix Dietrich, Rotem Mulayoff, Ronen Talmon, Ioannis G. Kerekidis, Spectral Discovery of Jointly Smooth Features for Multimodal Data, ArXiv, 2020, Available at: https://arxiv.org/abs/2004.04386

Jointly Smooth Functions learn common smooth functions from multimodal data.

In [ ]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
from scipy.sparse import SparseEfficiencyWarning, csr_matrix

# NOTE: make sure "path/to/datafold" is in sys.path or PYTHONPATH if datafold is not installed
import datafold.pcfold as pfold
from datafold.dynfold.jsf import JointlySmoothFunctions, JsfDataset

warnings.simplefilter("ignore", SparseEfficiencyWarning)

## Generate Parameters and Observations

We generate data for the parameters, the observations, and the effective parameters with the three functions below.

In [ ]:
def generate_parameters(_x, _y):
    return np.column_stack(
        [
            _x,
            _y,
        ]
    )


def generate_observations(_x, _z, div=5, mult=6):
    return np.column_stack(
        [
            (div / 2 * _z + _x / 2 + 2 / 3) * np.cos(mult * np.pi * _z) / 2,
            (div / 2 * _z + _x / 2 + 2 / 3) * np.sin(mult * np.pi * _z) / 2,
        ]
    )


def generate_points(n_samples):
    rng = np.random.default_rng(42)
    xyz = rng.uniform(low=-0.5, high=0.5, size=(n_samples, 3))
    x, y, z = (
        xyz[:, 0].reshape(-1, 1),
        xyz[:, 1].reshape(-1, 1),
        xyz[:, 2].reshape(-1, 1),
    )

    parameters = generate_parameters(x, y)
    effective_parameter = parameters[:, 0] + parameters[:, 1] ** 2
    observations = generate_observations(effective_parameter, z[:, 0], 2, 2)

    return parameters, observations, effective_parameter

## Plotting the parameters and observations colored by the effective parameter

The plots below illustrate the parameters and observations colored by the effective parameter, which we want to learn with the jointly smooth functions.

In [ ]:
def plot_parameters_and_observations(
    parameters: np.ndarray,
    observations: np.ndarray,
    effective_parameter: np.ndarray,
    n_samples_plot: int = 3000,
):
    n_samples = parameters.shape[0]
    idx_plot = np.random.permutation(n_samples)[:n_samples_plot]
    scatter_size = 10

    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(8, 3.5))

    ax[0].scatter(
        x=parameters[idx_plot, 0],
        y=parameters[idx_plot, 1],
        s=scatter_size,
        c=effective_parameter[idx_plot],
        cmap=plt.cm.Spectral,
    )
    ax[0].set_title("Parameters")
    ax[0].set_xlabel(r"$p_A$")
    ax[0].set_ylabel(r"$p_B$")

    ax[1].scatter(
        x=observations[idx_plot, 0],
        y=observations[idx_plot, 1],
        s=scatter_size,
        c=effective_parameter[idx_plot],
        cmap=plt.cm.Spectral,
    )
    ax[1].set_title("Observations")
    ax[1].set_xlabel(r"$z_A$")
    ax[1].set_ylabel(r"$z_B$")

    plt.show()

In [ ]:
n_samples = 6000
parameters, observations, effective_parameter = generate_points(n_samples)

plot_parameters_and_observations(parameters, observations, effective_parameter)

## The JsfDataset Class

`JsfDataset` does the slicing of the data. This is needed, as `.fit`, `.transform`, and `.fit_transform` accept a single data array `X`. Thus, the multimodal data is passed in as a single array and is then separated inside the methods. `JsfDataset.fit_transform` provides this splitting functionality. The constructor of `JsfDataset` expects: 
- a name
- a slice or list (the columns of `X` corresponding to this dataset)
- an optional kernel (default: `GaussianKernel`)
- optional dist_kwargs for the `PCManifold` created in `.fit_transform`

In [ ]:
cknn_delta = 1
cknn_k_neighbor = 50
kernel1 = pfold.kernels.ContinuousNNKernel(k_neighbor=cknn_k_neighbor, delta=cknn_delta)
kernel2 = pfold.kernels.ContinuousNNKernel(k_neighbor=cknn_k_neighbor, delta=cknn_delta)

X = np.column_stack([parameters, observations])


datasets = [
    JsfDataset(
        "parameters", slice(0, 2), kernel1, backend="scipy.kdtree", cut_off=1e-8
    ),
    JsfDataset(
        "observations", slice(2, 4), kernel2, backend="scipy.kdtree", cut_off=1e-8
    ),
]

## Fit JointlySmoothFunctions model

We create a `JointlySmoothFunctions` model and fit it to the data. Internally, `JointlySmoothFunctions` uses the datasets to split up the single input array `X` into the separate datasets.

In [ ]:
jsf = JointlySmoothFunctions(
    n_kernel_eigenvectors=100,
    n_jointly_smooth_functions=10,
    datasets=datasets,
    kernel_eigenvalue_cut_off=1e-8,
    eigenvector_tolerance=1e-10,
)

jsf.fit(X)

## The E0 Threshold

The `JointlySmoothFunctions` class also provides a method to calculate a threshold for the functions. The eigenvalues are depicted in blue and the threshold in red. 

In [ ]:
E0 = jsf.calculate_E0()

plt.plot(jsf.eigenvalues, ".-")
plt.axhline(y=E0, c="r")
plt.show()

## The Jointly Smooth Functions

Here, you can see that the last two functions (having eigenvalues below E0) are quite noisy.

In [ ]:
idx_plot = np.random.permutation(n_samples)

n_plots = 6
fig, ax = plt.subplots(1, n_plots, figsize=(n_plots * 3, 3), sharey=True)
for k in range(len(ax)):
    ax[k].scatter(
        effective_parameter[idx_plot],
        jsf.jointly_smooth_functions[idx_plot, k],
        c=jsf.jointly_smooth_functions[idx_plot, k],
        s=5,
        cmap=plt.cm.Spectral,
    )

## Plotting the parameters and observations colored by a jointly smooth function

The plots below illustrate the parameters and observations colored by a jointly smooth function.

In [ ]:
plot_parameters_and_observations(
    parameters=parameters,
    observations=observations,
    effective_parameter=jsf.jointly_smooth_functions[:, 1],
)

## Out-of-sample extension

Finally, `JointlySmoothFuctions` offers an out-of-sample extension via the `.transform` function. We create a new test set, transform the test set, and create the same plot as for the train set above.

In [ ]:
n_new_samples = 3000
new_parameters, new_observations, new_effective_parameter = generate_points(
    n_new_samples
)
new_X = np.column_stack([new_parameters, new_observations])

oos_jsfs = jsf.transform(new_X)

plot_parameters_and_observations(
    parameters=new_parameters,
    observations=new_observations,
    effective_parameter=oos_jsfs[:, 1],
)